# JQ Interactive Learning

Please first make sure that you have jq installed on your machine. You can check by running `jq --help | head -n 1` and seeing if the command is recognized. 

If you do not have it installed yet, you can install it with `brew install jq` on Mac. Please view the jq docs for installation instructions for other operating systems.

Check if jq is available in the notebook:

In [ ]:
%%bash
jq --help | head -n 1

In [ ]:
%%bash
echo '{"fruit":{"name":"apple","color":"green","price":1.20}}' | jq '.'

Let's look at a basic curl response without jq:

In [ ]:
%%bash
curl -s http://api.open-notify.org/iss-now.json

Apply jq to an API response - a common technique you will find in CICD scripts:

In [ ]:
%%bash
curl -s http://api.open-notify.org/iss-now.json | jq '.'

Return value based on provided key:

In [ ]:
%%bash
json='{"name": "apple","color": "green","price": 1.2}'
echo "$json" | jq '.color'

Basic json array example - this comes up with many APIs such as GitHub that returns the response as an array:

In [ ]:
%%bash
json='["Mike", "Serge", "Rachel", "Thomas", "David", "Adelaide", "Mika", "Julio", "Arvind"]'
echo "$json" | jq '.[]'

Let's look at how compact output can pretify arrays with larger nested objects - this can be useful for writing scripts that loop through arrays and apply changes based on the position of output:

In [ ]:
%%bash
json='[{"name": "apple","color": "green","price": 1.2},{"name": "banana","color": "yellow","price": 0.5},{"name": "kiwi","color": "green","price": 1.25}]'
echo "$json" | jq '.[]'

In [ ]:
%%bash
json='[{"name": "apple","color": "green","price": 1.2},{"name": "banana","color": "yellow","price": 0.5},{"name": "kiwi","color": "green","price": 1.25}]'
echo "$json" | jq -c '.[]'

Perhaps we would like to loop through an array of team members and sort the items alphabetically - notice the `-r` for raw, unquoted output:

In [ ]:
%%bash
json='[{"team1": "Mike"}, {"team1": "Serge"}, {"team1": "Rachel"}, {"team1": "Thomas"}, {"team1": "David"}, {"team1": "Adelaide"}, {"team1": "Mika"}, {"team1": "Julio"}, {"team1": "Arvind"}]'
echo "$json" | jq -r '.[] | .team1' | sort

How about indices with json arrays?

In [ ]:
%%bash
json='[{"team1": "Mike"}, {"team1": "Serge"}, {"team1": "Rachel"}, {"team1": "Thomas"}, {"team1": "David"}, {"team1": "Adelaide"}, {"team1": "Mika"}, {"team1": "Julio"}, {"team1": "Arvind"}]'
index=0
for row in $(echo "${json}" | jq -r '.[] | .team1'); do
    temp=$(echo "${json}" | jq -r '.['"$index"'] | .team1')
    echo "Row: ${index}, the team member at index ${index} is $temp"
    ((++index))
done

Finally, I will leave you with a practical example that checks that the build is running that latest commit.

We have the following bash script:
```
#!/bin/bash

OWNER="owner"
REPO="repo"

if [[ "$TRAVIS_EVENT_TYPE" == "push" && "$TRAVIS_BRANCH" == "master" ]]; then
    latest_master_commit=$(curl --silent -X GET \
        -H "Authorization: token $GH_TOKEN" \
        -H "Accept: application/vnd.github.v3+json" \
        https://github.com/api/v3/repos/${OWNER}/${REPO}/commits | jq -r '.[0] | .sha') 
    if [[ "$latest_master_commit" == "$TRAVIS_COMMIT" ]]; then
        echo "This build is running the latest commit, continuing to terraform apply..." 
        terraform apply -auto-approve
    else
        echo "This is not the latest commit, please merge master into your feature branch, resolve any conflicts, and re-run the build on the latest commit. The terraform state is remote, so if you re-run an old build it may cause infrastructure to be deleted because terraform will see infra in the state file that is not reflected in the IaC."
        exit 1  
    fi                                            
else
    echo "Not running protected-terraform-apply because this is not a push to master."
fi
```

Notice 

```
latest_master_commit=$(curl --silent -X GET \
        -H "Authorization: token $GH_TOKEN" \
        -H "Accept: application/vnd.github.v3+json" \
        https://github.com/api/v3/repos/${OWNER}/${REPO}/commits | jq -r '.[0] | .sha')
```
You can try it in the block below - make sure to generate a personal access token under your GitHub Developer Settings that has repo access and plug it in the variable along with the repo:

In [ ]:
%%bash
OWNER="owner"
REPO="repo"

GITHUB_PERSONAL_ACCESS_TOKEN="<insert>"

curl --silent -X GET \
        -H "Authorization: token $GITHUB_PERSONAL_ACCESS_TOKEN" \
        -H "Accept: application/vnd.github.v3+json" \
        https://github.com/api/v3/repos/"${OWNER}"/"${REPO}"/commits | jq -r '.[0] | .sha'